Miguel Angel Gomez<Br>

**Importando paquetes**

In [1]:
import pandas as pd
import nltk
nltk.download('averaged_perceptron_tagger')
import fitz #paquete para la lectura de pdf
import matplotlib.pyplot as plt
from stop_words import get_stop_words
import re
from nltk import SnowballStemmer
import spacy
import spacy_spanish_lemmatizer
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import wordnet
from wordcloud import WordCloud, ImageColorGenerator
from stop_words import get_stop_words
from collections import defaultdict
nlp=spacy.load('es_core_news_sm')
from spacy import displacy
import string
string.punctuation
from nltk.tokenize import word_tokenize


from sklearn import svm
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.naive_bayes import GaussianNB 
from sklearn.feature_selection import SelectKBest
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.cluster import KMeans

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Miguel Gomez\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


Importando dataframe

In [2]:
Noticias= pd.read_excel("Noticias.xlsx")
Noticias

,Columna1,Enlaces,Título,info,contenido,Etiqueta
0,0,https://www.eltiempo.com/agresion-contra-un-op...,Operador de grúa quedó inconsciente tras agres...,El conductor de una moto le lanzó el casco y p...,Las autoridades están buscando al conductor de...,colombia
1,1,https://www.eltiempo.com/archivo/documento/CMS...,"Usaquén, primera en infracciones por mal parqueo",La localidad ocupa el primer lugar en comparen...,"""Los andenes son para los peatones"", reclama e...",archivo
2,2,https://www.eltiempo.com/archivo/documento/CMS...,'Me atracaron y vi un arma que me heló la sang...,Un ciudadano relata cómo cuatro hombres lo rob...,A las 7 de la noche me había quedado de encont...,archivo
3,3,https://www.eltiempo.com/archivo/documento/CMS...,"Escoltas mal estacionados, dolor de cabeza de ...",Las zonas de restaurantes se convierten en par...,Atravesados. Eso es lo que se les pasa por la ...,archivo
4,4,https://www.eltiempo.com/archivo/documento/CMS...,Radicado primer proyecto que autorizaría union...,"El representante de 'la U', Miguel Gómez, dijo...",“Estamos proponiendo la figura de un contrato ...,archivo
...,...,...,...,...,...,...
14391,14391,https://www.eltiempo.com/vida/viajar/una-viaje...,Una viajera pensó que la aerolínea Viva Air le...,La mujer hizo la denuncia en redes. La aerolín...,En Twitter es tendencia 'FCAS' por una denunci...,vida
14392,14392,https://www.eltiempo.com/vida/viajar/un-dia-pe...,"Un día perfecto en Coco Cay, la isla privada d...",El proyecto Global Private Island Collection c...,La naviera internacional Royal Caribbean prese...,vida
14393,14393,https://www.eltiempo.com/vida/viajar/visitar-b...,Barú: el paraíso colombiano abierto para todos...,Cuatro hoteles y el Aviario Nacional lanzarán ...,¿Necesita razones para ir a Barú? A menos de u...,vida
14394,14394,https://www.eltiempo.com/vida/viajar/we-trade-...,"Con WeTrade, llega la hora LGBT a Bogotá",El 1 y 2 de noviembre se celebrará esta Feria ...,Se estima que la población LGTBI en Colombia e...,vida


Visualizando variables del dataset Noticias

In [3]:
Noticias.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14396 entries, 0 to 14395
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Columna1   14396 non-null  int64 
 1   Enlaces    14396 non-null  object
 2   Título     14378 non-null  object
 3   info       11483 non-null  object
 4   contenido  13738 non-null  object
 5   Etiqueta   14396 non-null  object
dtypes: int64(1), object(5)
memory usage: 674.9+ KB


**Funciones de limpieza de datos**

In [4]:
stop_words_español=get_stop_words('es') #obteniendo stop words en español

In [5]:
# función para quitar stopwords

def quitar_stopwords(texto):
    texto = " ".join([palabra for palabra in texto.split() if palabra not in stop_words_español])
    return texto

In [6]:
# funcion para quitar numeros de un txt

def quitar_numeros_txt(texto):
    texto = re.sub(r'\d+', '', texto)
    return texto


In [7]:
# Funcion para quitar quitar palabras despues de una palabra especifica
# esta función es de las mas importantes porque permite hacer la limpieza de las colillas de anuncios pegadas a las noticias 
def quitar_palabra_despues(texto, palabra):
    texto = texto.split(palabra, 1)[0]
    return texto

In [8]:
# Lista de palabras especiales para eliminar con la funcion quitarpalabra_despues
lista_de_palabras_patron = ["REDACCIÓN","Redacción Bogotá",
                            "PORTAFOLIO","Valle del Cauca 07:50",
                            "FRASES","ROMA","JUSTICIA", "BOCAS",
                            "REVISTA", "BOGOTÁ","carrusel",
                            "NACIÓN","CULTURA","ENTRETENIMIENTO","11:19"]

In [9]:
# delete worlds since a specific word

def quitar_palabras_despues(texto, lista_de_palabras_patron):
    for palabra in lista_de_palabras_patron:
        texto = quitar_palabra_despues(texto, palabra)
    return texto

In [10]:
# convertir en formato string Noticias['contenido']
Noticias['contenido']=Noticias['contenido'].astype(str)

In [11]:
#pasar a string la columna Noticias['contenido']
Noticias['contenido_cl']=Noticias['contenido']
# se aplica la funcion quitar palabras en un ciclo for 
for palabra in lista_de_palabras_patron:
    
    Noticias['contenido_cl']=Noticias['contenido_cl'].apply(lambda texto: quitar_palabra_despues(texto, palabra))

In [12]:
# SE CREA LA FUNCION QUE PERMITE CONTAR PALABRAS EN UNA COLUMNA DE UN DF
def contar_palabras(texto):
    texto = texto.split()
    return len(texto)

In [13]:
# aplicando la funcion que me permite contar el numero de palabras en la columna de contenido la cual no ha sido limpiada
Noticias['cantidad_de_palabras_contenido']=Noticias['contenido'].apply(lambda texto: contar_palabras(texto))

In [14]:
# aplicando la funcion para que me cuente el numero de palabras en la columna de contenido_cl la cual ya ha sido limpiada

Noticias['cantidad_de_palabras_contenido_cl']=Noticias['contenido_cl'].apply(lambda texto: contar_palabras(texto))

In [15]:
# comparar el numero de palabras en la columna de contenido_cl con el numero de palabras en la columna de contenido

Noticias['diferencia_palabras']=Noticias['cantidad_de_palabras_contenido']-Noticias['cantidad_de_palabras_contenido_cl']

In [16]:
# filtar diferencia de palabras mayor a 0

Noticias[Noticias['diferencia_palabras']==0]

,Columna1,Enlaces,Título,info,contenido,Etiqueta,contenido_cl,cantidad_de_palabras_contenido,cantidad_de_palabras_contenido_cl,diferencia_palabras
30,30,https://www.eltiempo.com/archivo/documento/CMS...,Sabella será presentado el sábado como nuevo s...,NaN,nan,archivo,nan,1,1,0
31,31,https://www.eltiempo.com/archivo/documento/CMS...,'Bolillo' Gómez se disculpa por agredir a muje...,NaN,nan,archivo,nan,1,1,0
32,32,https://www.eltiempo.com/archivo/documento/CMS...,"Lo de 'Bolillo', un golpe bajo para el fútbol ...",NaN,nan,archivo,nan,1,1,0
33,33,https://www.eltiempo.com/archivo/documento/CMS...,No para mala fortuna del DIM: William Arboleda...,NaN,nan,archivo,nan,1,1,0
34,34,https://www.eltiempo.com/archivo/documento/CMS...,Aplazado el partido del Tottenham con el Evert...,NaN,nan,archivo,nan,1,1,0
...,...,...,...,...,...,...,...,...,...,...
13109,13109,https://www.eltiempo.com/opinion/caricaturas/r...,Caricaturas,NaN,nan,opinion,nan,1,1,0
13110,13110,https://www.eltiempo.com/opinion/caricaturas/r...,Caricaturas,NaN,nan,opinion,nan,1,1,0
13111,13111,https://www.eltiempo.com/opinion/caricaturas/r...,Caricaturas,NaN,nan,opinion,nan,1,1,0
13112,13112,https://www.eltiempo.com/opinion/caricaturas/r...,Caricaturas,NaN,nan,opinion,nan,1,1,0


In [17]:
# sacar los datos nan de la columna de contenido

Noticias=Noticias[Noticias['contenido'].notna()]
Noticias

,Columna1,Enlaces,Título,info,contenido,Etiqueta,contenido_cl,cantidad_de_palabras_contenido,cantidad_de_palabras_contenido_cl,diferencia_palabras
0,0,https://www.eltiempo.com/agresion-contra-un-op...,Operador de grúa quedó inconsciente tras agres...,El conductor de una moto le lanzó el casco y p...,Las autoridades están buscando al conductor de...,colombia,Las autoridades están buscando al conductor de...,352,247,105
1,1,https://www.eltiempo.com/archivo/documento/CMS...,"Usaquén, primera en infracciones por mal parqueo",La localidad ocupa el primer lugar en comparen...,"""Los andenes son para los peatones"", reclama e...",archivo,"""Los andenes son para los peatones"", reclama e...",532,424,108
2,2,https://www.eltiempo.com/archivo/documento/CMS...,'Me atracaron y vi un arma que me heló la sang...,Un ciudadano relata cómo cuatro hombres lo rob...,A las 7 de la noche me había quedado de encont...,archivo,A las 7 de la noche me había quedado de encont...,513,409,104
3,3,https://www.eltiempo.com/archivo/documento/CMS...,"Escoltas mal estacionados, dolor de cabeza de ...",Las zonas de restaurantes se convierten en par...,Atravesados. Eso es lo que se les pasa por la ...,archivo,Atravesados. Eso es lo que se les pasa por la ...,488,380,108
4,4,https://www.eltiempo.com/archivo/documento/CMS...,Radicado primer proyecto que autorizaría union...,"El representante de 'la U', Miguel Gómez, dijo...",“Estamos proponiendo la figura de un contrato ...,archivo,“Estamos proponiendo la figura de un contrato ...,1024,918,106
...,...,...,...,...,...,...,...,...,...,...
14391,14391,https://www.eltiempo.com/vida/viajar/una-viaje...,Una viajera pensó que la aerolínea Viva Air le...,La mujer hizo la denuncia en redes. La aerolín...,En Twitter es tendencia 'FCAS' por una denunci...,vida,En Twitter es tendencia 'FCAS' por una denunci...,289,192,97
14392,14392,https://www.eltiempo.com/vida/viajar/un-dia-pe...,"Un día perfecto en Coco Cay, la isla privada d...",El proyecto Global Private Island Collection c...,La naviera internacional Royal Caribbean prese...,vida,La naviera internacional Royal Caribbean prese...,510,413,97
14393,14393,https://www.eltiempo.com/vida/viajar/visitar-b...,Barú: el paraíso colombiano abierto para todos...,Cuatro hoteles y el Aviario Nacional lanzarán ...,¿Necesita razones para ir a Barú? A menos de u...,vida,¿Necesita razones para ir a Barú? A menos de u...,436,336,100
14394,14394,https://www.eltiempo.com/vida/viajar/we-trade-...,"Con WeTrade, llega la hora LGBT a Bogotá",El 1 y 2 de noviembre se celebrará esta Feria ...,Se estima que la población LGTBI en Colombia e...,vida,Se estima que la población LGTBI en Colombia e...,701,600,101


In [18]:
Noticias.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14396 entries, 0 to 14395
Data columns (total 10 columns):
 #   Column                             Non-Null Count  Dtype 
---  ------                             --------------  ----- 
 0   Columna1                           14396 non-null  int64 
 1   Enlaces                            14396 non-null  object
 2   Título                             14378 non-null  object
 3   info                               11483 non-null  object
 4   contenido                          14396 non-null  object
 5   Etiqueta                           14396 non-null  object
 6   contenido_cl                       14396 non-null  object
 7   cantidad_de_palabras_contenido     14396 non-null  int64 
 8   cantidad_de_palabras_contenido_cl  14396 non-null  int64 
 9   diferencia_palabras                14396 non-null  int64 
dtypes: int64(4), object(6)
memory usage: 1.2+ MB


limpieando adicional

In [19]:
# quitamos las stopwords
Noticias['contenido_cl']=Noticias['contenido_cl'].apply(lambda texto: quitar_stopwords(texto))
Noticias['contenido_cl']

0        Las autoridades buscando conductor moto lanzó ...
1        "Los andenes peatones", reclama enfática *Carm...
2        A 7 noche quedado encontrar Boris, siempre cum...
3        Atravesados. Eso pasa cabeza residentes transe...
4        “Estamos proponiendo figura contrato civil uni...
                               ...                        
14391    En Twitter tendencia 'FCAS' denuncia hizo muje...
14392    La naviera internacional Royal Caribbean prese...
14393    ¿Necesita razones ir Barú? A menos hora Cartag...
14394    Se estima población LGTBI Colombia 3,3 millone...
14395    Con intención colombianos conozcan país aprove...
Name: contenido_cl, Length: 14396, dtype: object

In [20]:
# quitamos los numeros
Noticias['contenido_cl']=Noticias['contenido_cl'].apply(lambda texto: quitar_numeros_txt(texto))
Noticias['contenido_cl']

0        Las autoridades buscando conductor moto lanzó ...
1        "Los andenes peatones", reclama enfática *Carm...
2        A  noche quedado encontrar Boris, siempre cump...
3        Atravesados. Eso pasa cabeza residentes transe...
4        “Estamos proponiendo figura contrato civil uni...
                               ...                        
14391    En Twitter tendencia 'FCAS' denuncia hizo muje...
14392    La naviera internacional Royal Caribbean prese...
14393    ¿Necesita razones ir Barú? A menos hora Cartag...
14394    Se estima población LGTBI Colombia , millones ...
14395    Con intención colombianos conozcan país aprove...
Name: contenido_cl, Length: 14396, dtype: object

In [21]:
#quitar los signos de puntuacion
Noticias['contenido_cl']=Noticias['contenido_cl'].apply(lambda texto: texto.translate(str.maketrans('', '', string.punctuation)))
Noticias['contenido_cl']

0        Las autoridades buscando conductor moto lanzó ...
1        Los andenes peatones reclama enfática Carmenza...
2        A  noche quedado encontrar Boris siempre cumpl...
3        Atravesados Eso pasa cabeza residentes transeú...
4        “Estamos proponiendo figura contrato civil uni...
                               ...                        
14391    En Twitter tendencia FCAS denuncia hizo mujer ...
14392    La naviera internacional Royal Caribbean prese...
14393    ¿Necesita razones ir Barú A menos hora Cartage...
14394    Se estima población LGTBI Colombia  millones p...
14395    Con intención colombianos conozcan país aprove...
Name: contenido_cl, Length: 14396, dtype: object

Creando la columna de texto limpio y tokenizado

In [22]:
# function for tokenization 

def tokenizar(texto):
    tokens=word_tokenize(texto)
    return tokens

Noticias['contenido_cl_tk'] =   Noticias['contenido_cl'].apply(lambda x: tokenizar(x.lower()))
Noticias['contenido_cl_tk']

0        [las, autoridades, buscando, conductor, moto, ...
1        [los, andenes, peatones, reclama, enfática, ca...
2        [a, noche, quedado, encontrar, boris, siempre,...
3        [atravesados, eso, pasa, cabeza, residentes, t...
4        [“, estamos, proponiendo, figura, contrato, ci...
                               ...                        
14391    [en, twitter, tendencia, fcas, denuncia, hizo,...
14392    [la, naviera, internacional, royal, caribbean,...
14393    [¿necesita, razones, ir, barú, a, menos, hora,...
14394    [se, estima, población, lgtbi, colombia, millo...
14395    [con, intención, colombianos, conozcan, país, ...
Name: contenido_cl_tk, Length: 14396, dtype: object

### Stimming

In [23]:
doc = Noticias['contenido_cl']
text = doc.to_string()

In [24]:
texto_clean_columna=Noticias['contenido_cl'][0:1000]
# seleccionar las primeras 5 filas de la columna de texto limpio

In [25]:
texto_clean_columna

0      Las autoridades buscando conductor moto lanzó ...
1      Los andenes peatones reclama enfática Carmenza...
2      A  noche quedado encontrar Boris siempre cumpl...
3      Atravesados Eso pasa cabeza residentes transeú...
4      “Estamos proponiendo figura contrato civil uni...
                             ...                        
995                                                     
996                                                     
997                                                     
998    Hasta domingo noche solo jueza delegado Procur...
999    popular película vampiros Crepúsculo Amanecer ...
Name: contenido_cl, Length: 1000, dtype: object

In [26]:
def normalize(text):
    doc = nlp(text)
    words = [t.orth_ for t in doc if not t.is_punct | t.is_stop]
    lexical_tokens = [t.lower() for t in words if len(t) > 3 and     
    t.isalpha()]
    return lexical_tokens


In [27]:
Lista_texto_procesado=[]
for document in texto_clean_columna:
    Lista_texto_procesado.append(normalize(document))
    

In [28]:
len(Lista_texto_procesado)

1000

In [29]:
Lista_texto_procesado_2= [' '.join(k) for k in Lista_texto_procesado]

In [30]:
type(Lista_texto_procesado_2)

list

In [31]:
# pasar una lista a pandas.core.series.Series

df_texto_procesado_2=pd.Series(Lista_texto_procesado_2)

In [32]:
type(df_texto_procesado_2)

pandas.core.series.Series

In [33]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

sw=set(stopwords.words('spanish'))


[nltk_data] Downloading package stopwords to C:\Users\Miguel
[nltk_data]     Gomez\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


TFIDF

In [34]:
from sklearn.feature_extraction.text import TfidfVectorizer
modelo_de_conteo_2=TfidfVectorizer(stop_words=sw)

Vectorización

In [35]:
X_array_2=modelo_de_conteo_2.fit_transform(df_texto_procesado_2)
X_array_2

<1000x30304 sparse matrix of type '<class 'numpy.float64'>'
	with 121216 stored elements in Compressed Sparse Row format>

Cluster

In [36]:
modelo_agrupamiento=KMeans(n_clusters=10)
clusters=modelo_agrupamiento.fit_transform(X_array_2)
modelo_agrupamiento.labels_


array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 4,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 8, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 7, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,

In [37]:
Noticias_agrupdas=pd.DataFrame(df_texto_procesado_2)
Noticias_agrupdas['cluster']=modelo_agrupamiento.labels_

In [38]:
vocabulario=pd.DataFrame([list(modelo_de_conteo_2.vocabulary_.keys()),list(modelo_de_conteo_2.vocabulary_.values())]).T
vocabulario.columns=['Palabra','pos']
from IPython.core.display import HTML,display

for i in range(10):
    lista=modelo_agrupamiento.cluster_centers_[i].argsort()[-10:]
    display(HTML("<h1>CLUSTER "+str(i)+"</h1>"))
    display(HTML(vocabulario[vocabulario['pos'].isin(lista)].to_html()))

,Palabra,pos
37,hombre,14623
920,martínez,18384
2421,fernando,12743
2911,fotografías,13149
3042,sierra,26491
3167,manizales,18166
4967,vivo,29718
21686,arhuacos,2311
21688,zarwa,30144
21700,jano,16520


,Palabra,pos
1254,sencillo,26252
8614,regresa,24483
8615,colsubsidio,5993
8724,canción,4614
12384,exitoso,12169
12736,álbum,30195
12871,grabar,13919
14362,emisoras,10597
20061,site,26648
20062,lanzada,17004


,Palabra,pos
70,años,3222
159,pesos,21816
346,policía,22220
392,colombia,5956
682,bogotá,3854
686,país,21361
754,ciento,5528
758,presidente,22736
894,millones,18933
960,tiempo,28020


,Palabra,pos
392,colombia,5956
483,corte,7426
1589,gustavo,14192
2026,abogados,57
3222,reunirá,25225
8327,agente,859
11094,superados,27349
19189,designado,9032
28274,aclarados,319
28278,coagente,5764


,Palabra,pos
800,cambios,4512
2788,motor,19352
5834,camioneta,4543
6044,automóvil,3022
6129,automática,3017
6396,ruedas,25639
6677,funcionales,13352
6678,subaru,27103
6712,torque,28223
6757,mandos,18110


,Palabra,pos
346,policía,22220
430,velandia,29208
717,víctimas,29856
3077,banda,3344
3716,armados,2336
4325,coronel,7335
10286,finca,12913
11884,capturados,4730
21947,tabio,27580
21967,amarraron,1465


,Palabra,pos
621,alimentos,1272
932,hospitales,14717
1298,firma,12931
1859,carne,4851
3580,confianza,6582
16100,caballo,4283
22075,polonia,22243
25955,irlanda,16385
29787,hamburguesas,14342
29789,rangeland,23824


,Palabra,pos
17,video,29461
439,personaje,21746
1765,escritor,11427
3285,óscar,30287
10061,collazos,5937
13986,calumnia,4465
13992,duzán,10161
14019,condecorado,6501
14025,condecora,6498
14056,chambacú,5314


,Palabra,pos
779,llegar,17567
1019,madre,17937
1568,marcha,18273
2281,pereira,21603
2966,tierra,28030
10863,caminantes,4533
10871,abuela,141
10872,gonzalez,13873
10884,realizaron,23947
10891,guíame,14202


,Palabra,pos
894,millones,18933
930,locales,17658
943,obras,20206
952,abrir,99
1541,reciente,24063
1866,obra,20203
2315,pérez,23539
2779,plaza,22123
6452,necesarios,19764
8933,comerciantes,6085
